In [ ]:
import pandas as pd

# 파일 경로
xlsx_path = "E:\Traffic-Safety\서울시 자치구 읍면동별 연료별 자동차 등록현황(행정동)(25년03월).xlsx"

# 데이터 불러오기
df = pd.read_excel(xlsx_path, header=1)

# 구 정보 추출
df['구'] = df['Unnamed: 0'].where(df['Unnamed: 0'].str.contains("서울특별시", na=False))
df['구'] = df['구'].fillna(method='ffill')

# 차량 열
vehicle_columns = ['승용', '승합', '화물', '특수']
df[vehicle_columns] = df[vehicle_columns].apply(pd.to_numeric, errors='coerce')

# 구별 합산
gu_total = df.groupby('구')[vehicle_columns].sum()
gu_total['전체 차량 수'] = gu_total.sum(axis=1)

gu_total_simple = gu_total[['전체 차량 수']].reset_index()
# 엑셀로 저장
gu_total_simple


In [9]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import numpy as np
from collections import defaultdict
from dotenv import load_dotenv

# 한글 폰트 설정 (Windows 기준)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings("ignore", message="Workbook contains no default style, apply openpyxl's default")

def read_and_extract_city(file_path, year, city='서울', id_col_count=3):
    """
    엑셀 파일을 읽어와 3행을 다중 헤더로 사용하고, 
    범죄 항목(첫 id_col_count 열)에 forward-fill을 적용한 후,
    헤더 레벨 0(연도)가 지정한 year이고, 헤더 레벨 1(도시)가 지정한 city인 열만 추출하여 DataFrame 반환.
    """
    df = pd.read_excel(file_path, header=[0, 1, 2])
    df.iloc[:, :id_col_count] = df.iloc[:, :id_col_count].ffill()
    
    id_cols = df.columns[:id_col_count]
    other_cols = df.columns[id_col_count:]
    target_cols = [col for col in other_cols if str(col[0]).strip() == str(year) and col[1] == city]
    
    df_target = df.loc[:, list(id_cols) + target_cols].copy()
    df_target.columns = ['_'.join(map(str, col)).strip() for col in df_target.columns.values]
    
    return df_target

def read_and_extract_district(file_path, year, district, city='서울', id_col_count=3):
    """
    파일을 읽어와 3행 다중 헤더를 사용하고, 
    범죄 항목(첫 id_col_count 열)에 forward-fill을 적용한 후,
    헤더 레벨 0(연도)가 year, 레벨 1(도시)가 city, 레벨 2(구)가 district인 열만 선택하여 
    식별용 열과 함께 반환하는 함수.
    """
    df = pd.read_excel(file_path, header=[0, 1, 2])
    df.iloc[:, :id_col_count] = df.iloc[:, :id_col_count].ffill()
    
    id_cols = df.columns[:id_col_count]
    other_cols = df.columns[id_col_count:]
    
    target_cols = [col for col in other_cols 
                   if str(col[0]).strip() == str(year) 
                   and col[1] == city 
                   and str(col[2]).strip() == district]
    
    df_target = df.loc[:, list(id_cols) + target_cols].copy()
    df_target.columns = ['_'.join(map(str, col)).strip() for col in df_target.columns.values]
    
    return df_target

load_dotenv()

# 환경 변수 불러오기
file2023_path = os.getenv('FILE2023_PATH')
file_path22_21 = os.getenv('FILE_PATH22_21')
file_path19_20 = os.getenv('FILE_PATH19_20')
file_path17_18 = os.getenv('FILE_PATH17_18')
file_image_crime = os.getenv('File_image_crime')

# 서울 전체 데이터 추출 (연도별)
df2023 = read_and_extract_city(file2023_path, year=2023, city='서울')
df2022 = read_and_extract_city(file_path22_21, year=2022, city='서울')
df2021 = read_and_extract_city(file_path22_21, year=2021, city='서울')
df2020 = read_and_extract_city(file_path19_20, year=2020, city='서울')
df2019 = read_and_extract_city(file_path19_20, year=2019, city='서울')
df2018 = read_and_extract_city(file_path17_18, year=2018, city='서울')
df2017 = read_and_extract_city(file_path17_18, year=2017, city='서울')




In [10]:

# 'apple'이라는 단어가 포함된 행 필터링

three = '범죄별(3)_범죄별(3)_범죄별(3)'

filtered_2023 = df2023[df2023[three].str.contains('교통방해', case=False, na=False)]
filtered_2022 = df2022[df2022[three].str.contains('교통방해', case=False, na=False)]
filtered_2021 = df2021[df2021[three].str.contains('교통방해', case=False, na=False)]
filtered_2020 = df2020[df2020[three].str.contains('교통방해', case=False, na=False)]
filtered_2019 = df2019[df2019[three].str.contains('교통방해', case=False, na=False)]
filtered_2018 = df2018[df2018[three].str.contains('교통방해', case=False, na=False)]
filtered_2017 = df2017[df2017[three].str.contains('교통방해', case=False, na=False)]

filtered_2017

,범죄별(1)_범죄별(1)_범죄별(1),범죄별(2)_범죄별(2)_범죄별(2),범죄별(3)_범죄별(3)_범죄별(3),2017_서울_종로,2017_서울_중구,2017_서울_용산,2017_서울_성동,2017_서울_광진,2017_서울_동대문,2017_서울_중랑,...,2017_서울_강서,2017_서울_구로,2017_서울_금천,2017_서울_영등포,2017_서울_동작,2017_서울_관악,2017_서울_서초,2017_서울_강남,2017_서울_송파,2017_서울_강동
38,형법범,기타 형법범죄,교통방해,25,13,5,-,-,4,4,...,6,6,-,21,8,2,9,15,5,4


In [ ]:
import pandas as pd

filtered_2023.to_excel("filtered_2023.xlsx", index=False)  # index=False는 인덱스를 저장하지 않도록 설정
filtered_2022.to_excel("filtered_2022.xlsx", index=False)  # index=False는 인덱스를 저장하지 않도록 설정
filtered_2021.to_excel("filtered_2021.xlsx", index=False)  # index=False는 인덱스를 저장하지 않도록 설정
filtered_2020.to_excel("filtered_2020.xlsx", index=False)  # index=False는 인덱스를 저장하지 않도록 설정
filtered_2019.to_excel("filtered_2019.xlsx", index=False)  # index=False는 인덱스를 저장하지 않도록 설정
filtered_2018.to_excel("filtered_2018.xlsx", index=False)  # index=False는 인덱스를 저장하지 않도록 설정
filtered_2017.to_excel("filtered_2017.xlsx", index=False)  # index=False는 인덱스를 저장하지 않도록 설정


In [15]:
car_crime = r"E:\Traffic-Safety\범죄안전데이터\서울시 경찰청별 범죄 발생 및 검거 현황\filtered_2022-2017.xlsx"

df = pd.read_excel(car_crime)
df

,범죄별(1)_범죄별(1)_범죄별(1),범죄별(2)_범죄별(2)_범죄별(2),범죄별(3)_범죄별(3)_범죄별(3),종로,중구,용산,성동,광진,동대문,중랑,...,강서,구로,금천,영등포,동작,관악,서초,강남,송파,강동
0,형법범,기타 형법범죄,교통방해,25,13,5,-,-,4,4,...,6,6,-,21,8,2,9,15,5,4
1,형법범,기타 형법범죄,교통방해,2,8,2,3,6,5,5,...,3,3,1,9,2,3,6,4,-,1
2,형법범,기타 형법범죄,교통방해,4,-,4,3,-,2,-,...,4,1,-,3,4,2,6,5,7,4
3,형법범,기타 형법범죄,교통방해,5,2,-,-,4,2,1,...,2,2,1,2,8,3,4,3,1,2
4,형법범,기타 형법범죄,교통방해,6,3,8,1,2,5,2,...,3,2,1,4,2,7,5,5,2,7
5,형법범,기타 형법범죄,교통방해,9,10,5,1,3,4,1,...,1,7,1,1,-,2,3,6,-,-
6,형법범,기타 형법범죄,교통방해,5,3,3,2,2,3,7,...,4,4,2,2,4,3,7,1,6,1
7,소계,소계,소계,56,39,27,10,17,25,20,...,23,25,6,42,28,22,40,39,21,19


In [28]:
# 1. 소계 행만 추출
df_total = df[df['범죄별(3)_범죄별(3)_범죄별(3)'] == '소계']

# 2. melt로 자치구별 소계를 세로로 펼치기
df_result = df_total.melt(
    id_vars=['범죄별(1)_범죄별(1)_범죄별(1)', "범죄별(2)_범죄별(2)_범죄별(2)", '범죄별(3)_범죄별(3)_범죄별(3)'],
    var_name='자치구',
    value_name='소계'
)

# 3. 필요 컬럼만 정리
df_result = df_result[['자치구', '소계']]

# '자치구' 열 후처리 : 끝이 '구'가 아니면 '구'를 덧붙임
df_result['자치구'] = df_result['자치구'].apply(
    lambda x: x if str(x).endswith('구') else f'{x}구'
)
df_result

,자치구,소계
0,종로구,56
1,중구,39
2,용산구,27
3,성동구,10
4,광진구,17
5,동대문구,25
6,중랑구,20
7,성북구,17
8,강북구,13
9,도봉구,10


In [35]:
import json
import folium
import pandas as pd
from branca.colormap import LinearColormap

# 색상 단계 정의
colors = ['red', 'lightcoral', 'orange', 'blue', 'lightblue']
color_scale = LinearColormap(colors=colors,
                             vmin=df_result['소계'].min(),
                             vmax=df_result['소계'].max())

# GeoJSON 파일 로드
geojson_path = r'C:\Users\hyunj\Downloads\서울특별시.json'
with open(geojson_path, encoding='utf-8') as f:
    seoul_geo = json.load(f)

# GeoJSON에 'gu' 필드 추가
for feature in seoul_geo['features']:
    adm_nm = feature['properties']['adm_nm']  # 예: '서울특별시 성동구 행당동'
    gu_name = adm_nm.split()[1]  # '성동구'
    feature['properties']['gu'] = gu_name

# CCTV 데이터에서 자치구 필터링
geo_gu_list = {f['properties']['gu'] for f in seoul_geo['features']}
gu_counts_filtered = df_result[df_result['자치구'].isin(geo_gu_list)]

# folium 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

# 스타일 함수 정의 (색상 적용)
def style_function(feature):
    gu_name = feature['properties']['gu']
    value = gu_counts_filtered.loc[gu_counts_filtered['자치구'] == gu_name, '소계']
    if not value.empty:
        fill_color = color_scale(value.values[0])
    else:
        fill_color = 'gray'  # 데이터 없을 때
    return {
        'fillColor': fill_color,
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

# GeoJson으로 색상 직접 입히기
folium.GeoJson(
    seoul_geo,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['gu'], aliases=['자치구:'])
).add_to(m)

# 색상 범례 추가
color_scale.caption = '서울시 구별 CCTV 설치 수'
color_scale.add_to(m)

# 지도 저장
m.save("교통흐름방해.html")
